## Keras & Deep Learning
- book : [Deep Learning with Python by Francois Chollet](https://www.amazon.com/Deep-Learning-Python-Francois-Chollet/dp/1617294438)
- source code : [github](https://github.com/fchollet/deep-learning-with-python-notebooks)
- CogPsi #2 2019.3.26. Yoon Kyung Lee
- [git repository and notebooks](https://github.com/yoonlee78/cogpsi)

<img src = "./imgs/reuters.png" width = 250 >

<font color = blue> 3.5 뉴스 기사 분류: 다중 분류 문제 </font>
-----

### 차례

3.5.1. 로이터 데이터셋 <br>
3.5.2. 데이터 준비 <br>
3.5.3. 모델 구성 <br>
3.5.4. 훈련 검증 <br>
3.5.5. 새로운 데이터에 대해 예측하기 <br>
3.5.6. 레이블과 손실을 다루는 다른 방법 <br>
3.5.7. 충분히 중간층을 두어야 하는 이유 <br>
3.5.8 ~9. 정리 <br>


#### 지난 시간

완전 연결된 신경망을 사용하여 백터 입력을 2개의 클래스로 분류하는지 살펴보았다. 2개 이상의 클래스가 있을 때는 어떻게 할까?

#### 이번 시간

##### **<다중분류>** multiclass classification 

이 절에서는 로이터(Reuter)뉴스를 46개의 상호 배타적인 토픽으로 분류하는 신경망을 만들어보겠다. 

각 데이터 포인트가 정확히 하나의 범주로 분류되기 때문에 **단일 레이블 다중 분류 (single-label, multiclass classification)** 문제로 본다.*

----------

----------


*참고: 만약, 각 데이터 포인트가 여러개의 범주(or 토픽) 에 속할 수 있다면 **다중 레이블 다중 분류 (multi-label, multiclass classification)**  문제가 된다. 


### 3.5.1. 로이터 데이터셋

- 1986년에 로이터에서 공개한 짧은 뉴스기사와 토픽의 집합
- Reuters-21578, https://bit.ly/2JPwSa0
- 135개 토픽 중 샘플이 많은 것을 뽑아 간단하게 만들었으며 금융과 관련된 카테고리 
- IMDB, MNIST와 마찬가지로 이 데이터셋은 케라스에 포함되어있음. 


In [ ]:
from keras.datasets import reuters,models,layers
import numpy as np

In [ ]:
(train_data, train_labels),(test_data, test_labels) = reuters.load_data(num_words = 10000)

IMDB데이터셋처럼 num_words = 10000 매개 변수는 데이터에서 가장 자주 등장하는 단어 1만개로 제한함.

훈련 샘플과 테스트 샘플을 확인해보자 

In [ ]:
len(train_data)

In [ ]:
len(test_data)

IMDB 리뷰처럼 각 샘플은 정수(int)로된 리스트이다. 각 단어에 대한 인덱스 값이다. 


In [ ]:
train_data[10]

In [ ]:
#궁금한 경우 각 인덱스가 무슨 단어를 뜻하는지 디코딩 가능하다. 어떻게 디코딩하는지 알아보자. 
#로이터 데이터셋을 텍스트로 디코딩하기 

word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

#0,1,2는 '패딩','문서 시작','사전에 없음'을 위한 인덱스이므로 3을 뺀다. 
#제거되기 전의 모습을 보려면 join함수 안에 reverse_word_index.get(i)으로 대체. 


In [ ]:
decoded_newswire #책에 없음.

샘플에 연결된 레이블은 토픽의 인덱스로 0과 45 사이의 정수이다. 

In [ ]:
train_labels[10]

### 3.5.2 데이터 준비

데이터를 백터로 변환한다. 

**참고** 
IMDB와 로이터 데이터셋은 미리 전체 데이터셋의 단어를 고유한 정수 인덱스로 바꾼 후에 ```train_data```와 ```test_data```로 나누어 놓은 것임. 일반적으로는 ```train_data```에서 구축한 어휘 사전으로  ```test_data```를 변환한다. 이렇게 하는 이유는 실전에서 샘플에 어떤 텍스트가 들어 있을지 알 수 없기 때문에 ```test dataset```의 어휘를 이용하면 낙관적으로 테스트를 평가하는 셈이 되기 때문이다. 

In [ ]:
#데이터 인코딩
#import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

# 벡터화된 훈련 데이터
x_train = vectorize_sequences(train_data)
# 벡터화된 테스트 데이터
x_test = vectorize_sequences(test_data)

label -> vector 

- 레이블의 리스트를 정수 텐서로 변환하는 것
- 원-핫 인코딩 (범주형 인코딩_categorical encoding), 자세한 건 6.1에서. 
    -- 이 경우 레이블의 원-핫 인코딩은 각 레이블의 인덱스 자리는 1이고 나머지는 모두 0인 벡터. 
    

In [ ]:
#원-핫 인코딩
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

# 훈련 레이블 벡터 변환
one_hot_train_labels = to_one_hot(train_labels)

# 테스트 레이블 벡터 변환
one_hot_test_labels = to_one_hot(test_labels)


또는 Keras 내장 함수 사용


In [ ]:
#범주 변수를 벡터화하는 keras 내장함수 

from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

### 3.5.3. 모델 구성


영화 리뷰 분류(IMDB)문제와 비슷하게 짧은 텍스트를 분류하지만, 출력 클래스의 개수가 2개에서 46개로 늘어난 점이 다르다.즉, 출력 공간의 차원이 훨씬 커졌다. 

이전처럼 Dense 층을 쌓으면 각 층은 이전 층의 출력에서 제공한 정보만 사용할 수 있다. 한 층이 분류 문제에 필요한 일부 정보를 누락하면 그 다음 층에서 이를 복원할 방법이 없다. 각 층은 잠재적으로 정보의 병목(information bottleneck)이 될 수 있다. 

이전 예제에서 16차원을 가진 중간층을 사용했지만 16차원 공간은 46개의 클래스를 분류하기에 너무 제약이 많을 수 있다. 이렇게 규모가 작은 층은 유용한 정보를 완전히 잃게 될 수 있는 '정보의 병목 지점'처럼 동작할 수 있다. 

그러므로 좀 더 규모가 큰 64개의 유닛(층)을 사용한다. 

#### 1. 층 구성

In [ ]:
#from keras import models
#from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))


여기서, 
``` python
model.add(layers.Dense(46, activation='softmax'))
```

마지막 Dense층의 크기는 46인데, 각 입력 샘플에 대해 46차원의 벡터를 출력한다는 뜻. <br>
이 벡터의 각 원소(차원)는 각기 다른 출력 클래스가 인코딩 된 것.

또한, 마지막 층에 ```softmax ``` 활성화 함수가 사용됨. 각 입력 샘플마다 46개의 출력 클래스에 대한 확률 분포를 출력. 즉 46차원의 출력 벡터를 만들며 output[i]는 어떤 샘플이 클래스 i에 속할 확률. 46개의 값을 모두 더하면 1이 된다. 

#### 2. 손실 함수

이런 문제에 적합한 손실 함수는 **categorical_crossentropy**이다. 이 함수는 두 확률 분포 사이의 거리를 측정한다. 여기에서는 네트워크가 출력한 확률 분포와 진짜 레이블의 분포 사이의 거리이다. 두 분포 사이의 거리를 최소화하면 진짜 레이블에 가능한 가까운 출력을 내도록 모델을 훈련하게 된다. 

In [ ]:
#모델 컴파일

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


### 3.5.4. 훈련 검증

1. 검증 세트 

훈련 데이터에서 1,000개의 샘플을 따로 떼어서 검증 세트로 사용하겠다. 

In [ ]:
#검증 세트 준비하기

x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]


2. 모델 훈련 

In [ ]:
#20번의 에포크로 모델 훈련

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))


3. 손실과 정확도 곡선 

In [ ]:
#훈련과 검증 손실 그리기 

import matplotlib.pyplot as plt

In [ ]:
#훈련과 검증 손실 그리기 

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
#훈련과 검증 정확도 그리기

plt.clf()   # clear figure

acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


4. 평가

이 모델은 8번째 에포크 이후로 (즉, 9번째부터) 과대 적합 (overfitting)이 일어나는 것을 볼 수 있다. 9번의 에포크로 새로운 모델을 훈련하고 테스트 세트에서 평가해보자. 

5. 모델 재훈련 

In [ ]:
#모델을 처음부터 다시 훈련
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(partial_x_train,
          partial_y_train,
          epochs=8, #에포크를 8로 조정
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(x_test, one_hot_test_labels)

6. 최종 결과 

In [ ]:
results

대략 78%의 정확도를 달성함. 
균형 잡힌 이진 분류 문제에서 완전히 무작위로 분류하면 대략 50%의 정확도를 달성한다. 이 문제는 불균형한 데이터셋을 사용하므로 무작위로 분류하면 18% 정도 달성한다. 여기에 비하면 

아래 예시에 비하면 꽤 좋은 편이라고 할 수 있다. 


In [ ]:
import copy

test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
float(np.sum(np.array(test_labels) == np.array(test_labels_copy)))/ len(test_labels)

### 3.5.5. 새로운 데이터에 대해 예측하기 

모델 인스턴스의 predict 메서드는 46개의 토픽에 대한 확률 분포를 반환한다. 

테스트 데이터 전체에 대한 토픽을 예측해보자. 

In [ ]:
#새로운 데이터에 대해 예측하기 
predictions = model.predict(x_test)

In [ ]:
predictions[0].shape #각 항목은 길이가 46인 벡터

In [ ]:
np.sum(predictions[0]) # 벡터의 원소의 합은 1

가장 큰 값이 예측 클래스가 된다. 즉, 가장 확률이 높은 클래스이다. 

In [ ]:
np.argmax(predictions[0])

### 3.5.6. 레이블과 손실을 다루는 다른 방법

레이블을 인코딩하는 다른 방법을 앞서 소개했었다. 원 핫 인코딩 말고 다른 방법은 정수 텐서로 변환하는 것이다. 

train_labels와 test_labels는 정수(int)타입의 NumPy 배열이기 때문에 다시 np.array()함수를 사용할 필요는 없다. np.array() 함수는 np.asarray()함수와 동일하나 입력된 넘파이 배열의 복사본을 만들어 반환한다. 

In [ ]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

이 방식을 사용하려면 손실 함수 하나만 바꾸면 된다. 위에서 사용한 ```categorical_crossentropy``` 손실 함수는 레이블이 범주형 인코딩되어 있는 것을 가정하기 때문에 정수 레이블을 사용할 때는 ```sparse_categorical_crossentropy```를 사용한다. 

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

### 3.5.7. 충분히 큰 중간층을 두어야 하는 이유

앞서 언급한 바와 같이 마지막 출력이 46차원이기 때문에 중간층의 히든 유닛이 46개보다 많이 적어서는 안 된다. 46차원보다 훨씬 작은 중간층(예를 들어, 4차원)을 두면 정보의 병목이 어떻게 나타나는지 확인해보자

In [ ]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(partial_x_train,
          partial_y_train,
          epochs=20,
          batch_size=128,
          validation_data=(x_val, y_val))

검증 정확도의 최고 값은 <font color = red> **약 71%** </font>로 <font color = red> _8%정도 감소_</font>했다. <br> 이런 손실의 원인 대부분은 많은 정보 (클래스 46개의 분할 초평면을 복원하기에 충분한 정보)를 중간층의 저차원 표현 공간으로 압축하려고 했기 때문이다. 이 네트워크는 필요한 정보 대부분을 4차원 표현 안에 구겨 넣었지만 전부는 넣지 못한다. 

### 3.5.8. 선택: 추가 실험

- 더 크거나 작은 층을 사용해보자 (32개, 128개 유닛..등)
- 여기서는 2개의 은닉 층을 사용했다. 1개나 3개의 은닉 층을 사용해보세요. 

### 3.5.9. 정리 

- 다음은 이 예제에서 배운 것들이다. 

    - N개의 클래스로 데이터 포인트를 분류하려면 네트워크의 마지막 Dense 층의 크기는 N이어야 한다. 
    - 단일 레이블, 다중 분류 문제에서는 N개의 클래스에 대한 확률 분포를 출력하기 위해 softmax 활성화 함수를 사용해야 한다. 
    -  이런 문제에서는 항상 범주형 크로스엔트로피 (categorical_crossentropy)를 사용해야한다. 이 함수는 모델이 출력한 확률 분포와 타깃 분포 사이이 거리를 최소화 해준다. 
    - 다중 분류에서 레이블을 다루는 두 가지 방법 : 1) 레이블을 범주형 인코딩(cateogorical_crossentropy)하거나, 정수로 인코딩(sparse_categorical_crossentropy)한다. 
    - 많은 수의 범주를 분류할 때 중간층의 크기가 너무 작아 네트워크에 정보의 병목이 생기지 않도록 주의한다. 